In [3]:
#!/usr/bin/env python3
"""
Clean NDVI dataset, train RandomForest model for Option B, save artifacts.

Inputs:
 - /mnt/data/ndvi_filled_option_c_poly.xlsx

Outputs:
 - /mnt/data/ndvi_optionb_cleaned.csv
 - /mnt/data/rf_optionb_ndvi_model.joblib
 - /mnt/data/optionb_results_summary.json
"""

import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

INPUT_PATH = "/content/ndvi_filled_option_c_poly.xlsx"
CLEAN_CSV = "/content/ndvi_optionb_cleaned.csv"
MODEL_PATH = "/content/rf_optionb_ndvi_model.joblib"
RESULTS_JSON = "/content/optionb_results_summary.json"

def load_data(path=INPUT_PATH):
    df = pd.read_excel(path)
    df.columns = df.columns.str.lower().str.strip()
    return df

def clean_agronomic_years(df):
    # make sure numeric
    for col in ["planting_year", "harvest_year", "planting_month", "harvest_month"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # Find inconsistencies where planting_year > harvest_year
    mask = (df["planting_year"].notna()) & (df["harvest_year"].notna()) & (df["planting_year"] > df["harvest_year"])
    inconsistent = df[mask].copy()
    if not inconsistent.empty:
        # Fix: set harvest_year = planting_year (season alignment)
        df.loc[mask, "harvest_year"] = df.loc[mask, "planting_year"]
    return df, inconsistent

def prepare_modeling_df(df):
    # Filter potato rows and necessary features
    mask = df["product"].str.lower().str.contains("potato", na=False)
    df = df.loc[mask].copy()

    # Drop rows missing yield or mean_annual_ndvi
    df = df.dropna(subset=["yield", "mean_annual_ndvi"]).reset_index(drop=True)

    # Ensure numeric types
    df["area"] = pd.to_numeric(df["area"], errors="coerce").fillna(0)
    df["mean_annual_ndvi"] = pd.to_numeric(df["mean_annual_ndvi"], errors="coerce")

    # sort by harvest year chronologically for time split
    df = df.sort_values("harvest_year").reset_index(drop=True)
    return df

def build_features(df, encode_county=True, max_dummies=50):
    # Basic numeric features
    X_num = pd.DataFrame({
        "mean_annual_ndvi": df["mean_annual_ndvi"].astype(float),
        "area": df["area"].astype(float),
        "planting_month": df["planting_month"].fillna(0).astype(int)
    })

    # Optionally one-hot encode counties if not too many unique
    county_col = "admin_1"
    if encode_county and county_col in df.columns and df[county_col].nunique() <= max_dummies:
        dummies = pd.get_dummies(df[county_col].fillna("unknown"), prefix="adm1")
        X = pd.concat([X_num.reset_index(drop=True), dummies.reset_index(drop=True)], axis=1)
    else:
        X = X_num

    y = df["yield"].astype(float).values
    return X, y

def time_train_test_split(df, X, y, time_col="harvest_year"):
    years = pd.to_numeric(df[time_col], errors="coerce").astype("Int64")
    last_year = int(years.max())
    train_mask = years < last_year
    test_mask = years == last_year

    # If inadequate time-based train size, fallback to random 80/20
    if train_mask.sum() < 10 or test_mask.sum() < 5:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    else:
        X_train = X.loc[train_mask]
        y_train = y[train_mask.values]
        X_test = X.loc[test_mask]
        y_test = y[test_mask.values]
    return X_train, X_test, y_train, y_test, last_year

def train_rf(X_train, y_train, do_tune=False):
    rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    if not do_tune:
        rf.fit(X_train, y_train)
        return rf, None
    # Randomized search with TimeSeriesSplit
    param_dist = {
        "n_estimators": [200, 300, 500],
        "max_depth": [5, 10, 15, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    }
    tscv = TimeSeriesSplit(n_splits=5)
    rnd = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=20, cv=tscv, scoring="neg_mean_absolute_error", n_jobs=-1, random_state=42)
    rnd.fit(X_train, y_train)
    best = rnd.best_estimator_
    return best, rnd

def evaluate(y_test, y_pred):
    if len(y_test) == 0:
        return {"mae": None, "rmse": None, "r2": None}
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calculate RMSE manually
    r2 = r2_score(y_test, y_pred)
    return {"mae": float(mae), "rmse": float(rmse), "r2": float(r2)}

def main():
    print("Loading dataset:", INPUT_PATH)
    df = load_data(INPUT_PATH)

    print("Cleaning agronomic years...")
    df_clean, inconsistent_rows = clean_agronomic_years(df)
    print("Inconsistent rows corrected:", inconsistent_rows.shape[0])

    print("Saving cleaned CSV to:", CLEAN_CSV)
    df_clean.to_csv(CLEAN_CSV, index=False)

    print("Preparing modeling dataset...")
    mdl = prepare_modeling_df(df_clean)
    print("Modeling rows:", mdl.shape[0])

    X, y = build_features(mdl, encode_county=True, max_dummies=100)
    X_train, X_test, y_train, y_test, last_year = time_train_test_split(mdl, X, y)

    print("Train rows:", X_train.shape[0], "Test rows:", X_test.shape[0], "Last year:", last_year)

    print("Training RandomForest (no hyperparameter tuning by default)...")
    rf, rnd_search = train_rf(X_train, y_train, do_tune=False)

    print("Predicting on test set...")
    y_pred = rf.predict(X_test) if X_test.shape[0] > 0 else np.array([])

    results = evaluate(y_test, y_pred)
    print("Evaluation:", results)

    # Save model
    print("Saving model to:", MODEL_PATH)
    joblib.dump(rf, MODEL_PATH)

    summary = {
        "input_path": INPUT_PATH,
        "clean_csv": CLEAN_CSV,
        "model_path": MODEL_PATH,
        "n_rows_original": int(pd.read_excel(INPUT_PATH).shape[0]),
        "n_rows_cleaned": int(df_clean.shape[0]),
        "n_rows_modeling": int(mdl.shape[0]),
        "unique_counties": int(mdl["admin_1"].nunique()),
        "last_harvest_year_used_as_test": int(last_year),
        "train_rows": int(X_train.shape[0]),
        "test_rows": int(X_test.shape[0]),
        "mae": results["mae"],
        "rmse": results["rmse"],
        "r2": results["r2"]
    }

    with open(RESULTS_JSON, "w") as f:
        json.dump(summary, f, indent=2)

    print("Saved results summary to:", RESULTS_JSON)
    print("Done.")

if __name__ == "__main__":
    main()


Loading dataset: /content/ndvi_filled_option_c_poly.xlsx
Cleaning agronomic years...
Inconsistent rows corrected: 562
Saving cleaned CSV to: /content/ndvi_optionb_cleaned.csv
Preparing modeling dataset...
Modeling rows: 556
Train rows: 527 Test rows: 29 Last year: 2021
Training RandomForest (no hyperparameter tuning by default)...
Predicting on test set...
Evaluation: {'mae': 1.7613868040144838, 'rmse': 2.5331983109728173, 'r2': 0.5183011483152505}
Saving model to: /content/rf_optionb_ndvi_model.joblib
Saved results summary to: /content/optionb_results_summary.json
Done.
